In [37]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.multidatamodel import MultiDataModel

### Script to train and deploy model to sagemaker
Requires training data to be generated and uploaded to s3
Deploys to a serverless endpoint that scales on demand, tests show cold start is ~6s, after that subsecond

In [38]:
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

AWS Region: us-east-2
RoleArn: arn:aws:iam::257056996471:role/cb-sagemaker


In [39]:
# Train my estimator
# from sagemaker import SKLearn

sklearn_estimator = SKLearn(entry_point='train_deploy_ds_all_assessor_model.py',
                            instance_type='ml.m5.xlarge',
                            role=role,
                            framework_version='1.0-1')
                            
sklearn_estimator.fit({'train': 's3://cb-analytics-us-east-2-prd/sagemaker/training_data/20221130_211605_all_assessor_model.csv'})

2022-12-02 18:55:04 Starting - Starting the training job...
2022-12-02 18:55:28 Starting - Preparing the instances for trainingProfilerReport-1670007304: InProgress
......
2022-12-02 18:56:28 Downloading - Downloading input data...
2022-12-02 18:56:52 Training - Training image download completed. Training in progress..2022-12-02 18:57:03,909 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-12-02 18:57:03,912 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-02 18:57:03,920 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-12-02 18:57:04,063 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-02 18:57:04,074 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-02 18:57:04,084 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-02 18:57:04,093 sagemaker-trainin

### Serverless deploy

In [40]:
from time import gmtime, strftime

model_name = "ds-rec-hrs-v1-serverless-" + strftime("%Y%m%d-%H%M%S", gmtime())
print("Model name: " + model_name)

Model name: ds-rec-hrs-v1-serverless-20221202-185746


In [41]:
serverless_conf = sagemaker.serverless.serverless_inference_config.ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=100
)

In [42]:
predictor = sklearn_estimator.deploy(serverless_inference_config=serverless_conf,
                                     endpoint_name='ep-' + model_name,
                                     model_name=model_name
                                    )

------!

In [43]:
predictor.endpoint_name

'ep-ds-rec-hrs-v1-serverless-20221202-185746'

### Attempt at multi model endpoint

In [21]:
model = sklearn_estimator.create_model()

In [22]:
# This is where our MME will read models from on S3.
# model_data_prefix = f"s3://{BUCKET}/{DATA_PREFIX}/{MULTI_MODEL_ARTIFACTS}/"
model_data_prefix = "s3://sagemaker-us-east-2-257056996471/models/ds_rec_hrs_model/multi_model_artifacts"
print(model_data_prefix)

s3://sagemaker-us-east-2-257056996471/models/ds_rec_hrs_model/multi_model_artifacts


In [23]:
mme = MultiDataModel(
    name="ds-rec-hrs-model",
    model_data_prefix=model_data_prefix,
    model=model,  # passing our model
    sagemaker_session=sagemaker.Session(),
)

In [24]:
predictor = mme.deploy(
    initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name="ds-rec-hrs-model-20221130"
)

-----!

In [17]:
artifact_path = sklearn_estimator.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]
model_name = artifact_path.split("/")[-4] + ".tar.gz"
# This is copying over the model artifact to the S3 location for the MME.
mme.add_model(model_data_source=artifact_path, model_data_path=model_name)

ClientError: An error occurred (AccessDenied) when calling the CopyObject operation: Access Denied

In [5]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = sklearn_estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1)

# `data` is a NumPy array or a Python list.
# `response` is a NumPy array.
# response = predictor.predict(data)

-------!

In [6]:
predictor.endpoint_name

'sagemaker-scikit-learn-2022-03-30-17-43-42-670'

In [7]:
print('hello')

hello


In [ ]:
raise "STOP"

In [3]:
prefix = 'sagemaker'
model_name = 'hgbr_model_v2_mom_stddos_dem_12m_target'
s3_model_output_location ='s3://{}/{}/{}'.format('cb-analytics-exports-us-east-2-prd', prefix, model_name)
inputs_bucket_name = 'cb-analytics-us-east-2-prd'